# HCAN 361 Final Project: Spring 2021

## Emergent Care in  Hospitals within the Deep South US

In [ ]:
USE HCAN361_FinalProj

--The query below restricts analyses to census tracts in Alabama, Mississppi, and Georgia
--It also categorizes rurality and socioeconomic deprivation
select [GEOID_Data], [State_Name],[County_Name], [HRSA_RURAL_DESC],
case when [Median_ADI_NatRank]<=25 then '02: Upper Class'
when ([Median_ADI_NatRank]>25 and [Median_ADI_NatRank]<=75) then '01: Middle Class'
when [Median_ADI_NatRank]>75 then '00: Lower Class'
else '99: Not Ranked' end as [ADI]
into #ADI_RANK
from [dbo].[US_Census_Tract_Charac]
where ([State_Name] = 'Alabama' or [State_Name] = 'Mississippi' or [State_Name]= 'Georgia')

--Preview of Table
select top 10 *
from #ADI_RANK

(3814 rows affected)

(10 rows affected)

Total execution time: 00:00:00.159

GEOID_Data,State_Name,County_Name,HRSA_RURAL_DESC,ADI
14000US01001020100,Alabama,Autauga,00: Urban,01: Middle Class
14000US01001020200,Alabama,Autauga,00: Urban,01: Middle Class
14000US01001020300,Alabama,Autauga,00: Urban,01: Middle Class
14000US01001020400,Alabama,Autauga,00: Urban,01: Middle Class
14000US01001020500,Alabama,Autauga,00: Urban,01: Middle Class
14000US01001020600,Alabama,Autauga,00: Urban,01: Middle Class
14000US01001020700,Alabama,Autauga,00: Urban,01: Middle Class
14000US01001020801,Alabama,Autauga,00: Urban,02: Upper Class
14000US01001020802,Alabama,Autauga,00: Urban,01: Middle Class
14000US01001020900,Alabama,Autauga,00: Urban,01: Middle Class


In [ ]:
--The query below focuses on estimating travel time and provider ID for the cloest hospital with emergent care services
SELECT
#ADI_RANK.GEOID_Data, STATE_NAME, COUNTY_NAME, HRSA_RURAL_DESC, ADI AS 'ADI_RANK', ISNULL(MIN(TOTAL_TRAVELTIME),361) AS Closest_Provider
INTO #CLOSEST_PROVIDERS
FROM #ADI_RANK
LEFT JOIN Travel_Matrix 
ON Travel_Matrix.GEOID_Data = #ADI_RANK.GEOID_Data
WHERE Travel_Matrix.Provider_ID IN
(SELECT ID FROM AS19SVC3 WHERE URGCCHOS='1')
GROUP BY #ADI_RANK.GEOID_Data, State_Name, County_Name, HRSA_RURAL_DESC, ADI
ORDER BY #ADI_RANK.GEOID_Data

SELECT DISTINCT A.*, B.Provider_ID
INTO #CLOSEST_PROVIDERS_V2
FROM #CLOSEST_PROVIDERS A
LEFT JOIN
Travel_Matrix B
	ON A.GEOID_Data=b.GEOID_Data AND a.Closest_Provider=B.Total_TravelTime
AND B.Provider_ID IN (SELECT ID FROM AS19SVC3 WHERE URGCCHOS='1') 

--Preview of Table
select top 10 *
from #CLOSEST_PROVIDERS_V2

(3809 rows affected)

(3809 rows affected)

(10 rows affected)

Total execution time: 00:00:10.105

GEOID_Data,STATE_NAME,COUNTY_NAME,HRSA_RURAL_DESC,ADI_RANK,Closest_Provider,Provider_ID
14000US01001020300,Alabama,Autauga,00: Urban,01: Middle Class,21.55145796874138,6530695
14000US01001020802,Alabama,Autauga,00: Urban,01: Middle Class,24.944714472723007,6530695
14000US01003010906,Alabama,Baldwin,00: Urban,01: Middle Class,14.612707383294344,6530446
14000US01003011102,Alabama,Baldwin,00: Urban,01: Middle Class,15.166547070071065,6530446
14000US01003011201,Alabama,Baldwin,00: Urban,01: Middle Class,25.94520730719363,6530446
14000US01003011202,Alabama,Baldwin,00: Urban,01: Middle Class,24.239091846571522,6530446
14000US01003011601,Alabama,Baldwin,01: Rural,01: Middle Class,13.702793681395502,6530446
14000US01005950200,Alabama,Barbour,01: Rural,00: Lower Class,88.29021003210619,6530695
14000US01005950800,Alabama,Barbour,01: Rural,01: Middle Class,88.47903372973727,6380486
14000US01005950900,Alabama,Barbour,01: Rural,01: Middle Class,89.8892561373778,6380486


In [ ]:
--The query below focuses on estimating travel time and provider ID regardless of whether hospital has emergent care services
select A.*
INTO #ADDURG
from
(
SELECT
GEOID_Data,Total_TravelTime, Provider_ID,
  ROW_NUMBER() OVER(PARTITION BY GEOID_Data ORDER BY Total_TravelTime ASC) 
    AS [Rank]
FROM HCAN361_FinalProj.dbo.Travel_Matrix
) as A
where [Rank]=1
order by GEOID_Data,[Rank]


--This query restricts the above table to Alabama, Mississippi, and Georgia
SELECT #ADI_RANK.GEOID_Data, State_Name, County_Name, Provider_ID as 'Closest_Provider_ID', ISNULL(Total_TravelTime,361) AS 'Closest_Provider_Minutes', HRSA_RURAL_DESC, ADI AS 'ADI_RANK'
INTO #NEWADDURG
FROM #ADI_RANK
LEFT JOIN #ADDURG
ON #ADDURG.GEOID_Data = #ADI_RANK.GEOID_Data
ORDER BY State_Name, #ADDURG.GEOID_Data

--JOIN [dbo].[AS19SVC3].URGCCHOS ONTO ABOVE TABLE
--to ID whether closest hospital has emergent care services
SELECT 
GEOID_Data, State_Name, County_Name, 
Closest_Provider_ID, isnull(cast(URGCCHOS as int),-1) as URGCCHOS, Closest_Provider_Minutes, 
HRSA_RURAL_DESC, ADI_RANK
INTO #REALFINALFILTER
FROM #NEWADDURG
LEFT JOIN AS19SVC3
ON #NEWADDURG.Closest_Provider_ID = AS19SVC3.ID
ORDER BY State_Name, GEOID_Data

--Preview of Table
select top 10 *
from #REALFINALFILTER

(73908 rows affected)

(3814 rows affected)

(3814 rows affected)

(10 rows affected)

Total execution time: 00:00:42.410

GEOID_Data,State_Name,County_Name,Closest_Provider_ID,URGCCHOS,Closest_Provider_Minutes,HRSA_RURAL_DESC,ADI_RANK
14000US28067950700,Mississippi,Jones,6540215,1,6.358445324785784,01: Rural,00: Lower Class
14000US28123020500,Mississippi,Scott,6540235,0,3.43769867670021,01: Rural,01: Middle Class
14000US28123020600,Mississippi,Scott,6540235,0,13.336179541601274,01: Rural,00: Lower Class
14000US28123020100,Mississippi,Scott,6540235,0,14.715777999206649,01: Rural,00: Lower Class
14000US28083950200,Mississippi,Leflore,6540280,0,19.11947353746259,01: Rural,00: Lower Class
14000US28083950300,Mississippi,Leflore,6540280,0,20.98816418341398,01: Rural,00: Lower Class
14000US28083950600,Mississippi,Leflore,6540280,0,3.315992097420708,01: Rural,01: Middle Class
14000US28083950400,Mississippi,Leflore,6540280,0,10.68491158826764,01: Rural,00: Lower Class
14000US28083950700,Mississippi,Leflore,6540280,0,3.487723271236971,01: Rural,00: Lower Class
14000US28083950900,Mississippi,Leflore,6540280,0,5.444767519530529,01: Rural,00: Lower Class


In [ ]:
--This is the final table used for mapping
select A.GEOID_Data, A.GEOID, A.STATEFP, A.State_Name, A.COUNTYFP, A.County_Name
      ,A.ADI_RANK, A.HRSA_RURAL_DESC
	  ,case when A.URGCCHOS=1 then '01: Closest Hospital Has Emergent Care'
	        when A.URGCCHOS=0 then '00: Closest Hospital No Emergent Care'
			when A.URGCCHOS=-1 then '99: Closest Hospital Did Not Respond to Survey Question'
			end as [Emergent Care]
	  ,A.[CLOSEST HOSPITAL ID],A.[CLOSEST HOSPITAL TIME], B.Provider_ID as [CLOSEST EMERGENT ID], ISNULL(B.Closest_Provider,361)  as [CLOSEST EMERGENT TIME]
into #Final_Map_Table
from
(
SELECT #REALFINALFILTER.GEOID_Data, #REALFINALFILTER.State_Name, #REALFINALFILTER.County_Name, Closest_Provider_ID AS [CLOSEST HOSPITAL ID], URGCCHOS, Closest_Provider_Minutes AS [CLOSEST HOSPITAL TIME], 
#REALFINALFILTER.HRSA_RURAL_DESC, ADI_RANK, STATEFP, COUNTYFP, TRACTCE, GEOID
FROM #REALFINALFILTER
INNER JOIN US_Census_Tract_Charac
ON #REALFINALFILTER.GEOID_Data = US_Census_Tract_Charac.GEOID_DATA
) as A
LEFT JOIN
(
select *
from #CLOSEST_PROVIDERS_V2
) B
	ON A.GEOID_Data=B.GEOID_Data
ORDER BY A.GEOID_Data;

--Preview of Table
select top 10 *
from #Final_Map_Table

(3814 rows affected)

(10 rows affected)

Total execution time: 00:00:00.088

GEOID_Data,GEOID,STATEFP,State_Name,COUNTYFP,County_Name,ADI_RANK,HRSA_RURAL_DESC,Emergent Care,CLOSEST HOSPITAL ID,CLOSEST HOSPITAL TIME,CLOSEST EMERGENT ID,CLOSEST EMERGENT TIME
14000US13121010121,13121010121,13,Georgia,121,Fulton,01: Middle Class,00: Urban,99: Closest Hospital Did Not Respond to Survey Question,6380511,5.476301888766582,6380171,8.064314309433087
14000US13121010122,13121010122,13,Georgia,121,Fulton,02: Upper Class,00: Urban,99: Closest Hospital Did Not Respond to Survey Question,6380511,2.340156568033269,6380171,9.394262724821559
14000US13121010123,13121010123,13,Georgia,121,Fulton,01: Middle Class,00: Urban,99: Closest Hospital Did Not Respond to Survey Question,6380511,4.927888700085125,6380171,9.127878067454445
14000US13121010204,13121010204,13,Georgia,121,Fulton,02: Upper Class,00: Urban,00: Closest Hospital No Emergent Care,6380200,11.75685233379663,6380171,11.846503691558375
14000US13121010205,13121010205,13,Georgia,121,Fulton,02: Upper Class,00: Urban,99: Closest Hospital Did Not Respond to Survey Question,6380511,7.936644387259558,6380171,9.957089643160238
14000US13121010206,13121010206,13,Georgia,121,Fulton,02: Upper Class,00: Urban,00: Closest Hospital No Emergent Care,6380200,11.939407916164601,6380171,12.029059273926347
14000US13121010208,13121010208,13,Georgia,121,Fulton,02: Upper Class,00: Urban,99: Closest Hospital Did Not Respond to Survey Question,6380511,11.54953774010351,6380171,15.7246107516822
14000US13121010209,13121010209,13,Georgia,121,Fulton,01: Middle Class,00: Urban,99: Closest Hospital Did Not Respond to Survey Question,6380511,8.933029251926168,6380171,15.410913916464768
14000US13121010210,13121010210,13,Georgia,121,Fulton,02: Upper Class,00: Urban,99: Closest Hospital Did Not Respond to Survey Question,6380511,7.146663936926637,6380171,13.161362351338422
14000US13121010211,13121010211,13,Georgia,121,Fulton,02: Upper Class,00: Urban,00: Closest Hospital No Emergent Care,6380200,11.467687641622648,6380171,11.557338999384392


In [ ]:
--The query below are for showing provider locations.
--Will clip the extent later...including border states for areas near state borders
SELECT ID, LAT, LONG, MNAME
INTO #PROV_LOCS
FROM dbo.AS19DEM
WHERE ID IN
(SELECT ID FROM AS19SVC3 WHERE URGCCHOS='1')
and MSTATE in
('AL','MS','GA','TN','FL','SC','LA','AR')


(103 rows affected)

Total execution time: 00:00:00.057

In [ ]:
--Queries to build a few tables
--By Rurality using Average
select 
 State_Name
,avg(case when HRSA_RURAL_DESC='00: Urban' then [Closest Hospital Time] end) as [Avg Closest Hospital Time Urban]
,avg(case when HRSA_RURAL_DESC<>'00: Urban' then [Closest Hospital Time] end) as [Avg Closest Hospital Time Rural]
,avg([Closest Hospital Time]) as [Avg Closest Hospital Time Statewide]
,avg(case when HRSA_RURAL_DESC='00: Urban' then [CLOSEST EMERGENT TIME] end) as [Avg Closest Emergent Hospital Time Urban]
,avg(case when HRSA_RURAL_DESC<>'00: Urban' then [CLOSEST EMERGENT TIME] end) as [Avg Closest Emergent Hospital Time Rural]
,avg([CLOSEST EMERGENT TIME]) as [Avg Closest Emergent Hospital Time Statewide]
from #Final_Map_Table
group by State_Name
order by State_Name;

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.042

State_Name,Avg Closest Hospital Time Urban,Avg Closest Hospital Time Rural,Avg Closest Hospital Time Statewide,Avg Closest Emergent Hospital Time Urban,Avg Closest Emergent Hospital Time Rural,Avg Closest Emergent Hospital Time Statewide
Alabama,12.001998694230155,15.845634438984353,13.215949695748643,39.26094251409543,52.979408564157964,43.5937010548857
Georgia,12.031660785919737,16.122799793641185,12.993670618614036,31.983194735237216,53.3475155096493,37.006902464314315
Mississippi,11.072901073470044,12.720873058815185,12.038354540788223,40.91162996002498,70.6290469698773,58.32138179260404


In [ ]:
--By Rurality using Median
select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then [Closest Hospital Time] end) OVER (PARTITION BY State_Name) AS [Median Closest Hospital Time Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then [Closest Hospital Time] end) OVER (PARTITION BY State_Name) AS [Median Closest Hospital Time Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY [Closest Hospital Time]) OVER (PARTITION BY State_Name) AS [Median Closest Hospital Time Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then [CLOSEST EMERGENT TIME] end) OVER (PARTITION BY State_Name) AS [Median Closest Emergent Hospital Time Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then [CLOSEST EMERGENT TIME] end) OVER (PARTITION BY State_Name) AS [Median Closest Emergent Hospital Time Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY [CLOSEST EMERGENT TIME]) OVER (PARTITION BY State_Name) AS [Median Closest Emergent Hospital Time Statewide]
from #Final_Map_Table
order by State_Name;

(3 rows affected)

Total execution time: 00:00:00.196

State_Name,Median Closest Hospital Time Urban,Median Closest Hospital Time Rural,Median Closest Hospital Time Statewide,Median Closest Emergent Hospital Time Urban,Median Closest Emergent Hospital Time Rural,Median Closest Emergent Hospital Time Statewide
Alabama,10.326237108115452,14.916436449722534,11.601299666024637,37.62589378604012,51.688992143382556,41.907697256618754
Georgia,10.602925301316727,13.440875992043281,11.037669217700714,22.96642044877656,48.09893676064021,27.26978209230606
Mississippi,9.594606326957217,12.441737739275291,11.154917010340867,30.082025102384506,70.02629485510955,59.19426063772927


In [ ]:
--Avg by ADI_Rank (Note that non ranked tracts are included in statewide findings)
select 
 State_Name
,avg(case when ADI_Rank='00: Lower Class' then [Closest Hospital Time] end) as [Avg Closest Hospital Time Low Class]
,avg(case when ADI_Rank='01: Middle Class' then [Closest Hospital Time] end) as [Avg Closest Hospital Time Middle Class]
,avg(case when ADI_Rank='02: Upper Class' then [Closest Hospital Time] end) as [Avg Closest Hospital Time Upper Class]
,avg([Closest Hospital Time]) as [Avg Closest Hospital Time Statewide]
,avg(case when ADI_Rank='00: Lower Class' then [CLOSEST EMERGENT TIME] end) as [Avg Closest Emergent Hospital Time Low Class]
,avg(case when ADI_Rank='01: Middle Class' then [CLOSEST EMERGENT TIME] end) as [Avg Closest Emergent Hospital Time Middle Class]
,avg(case when ADI_Rank='02: Upper Class' then [CLOSEST EMERGENT TIME] end) as [Avg Closest Emergent Hospital Time Upper Class]
,avg([CLOSEST EMERGENT TIME]) as [Avg Closest Emergent Hospital Time Statewide]
from #Final_Map_Table
group by State_Name
Order by State_Name;

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.045

State_Name,Avg Closest Hospital Time Low Class,Avg Closest Hospital Time Middle Class,Avg Closest Hospital Time Upper Class,Avg Closest Hospital Time Statewide,Avg Closest Emergent Hospital Time Low Class,Avg Closest Emergent Hospital Time Middle Class,Avg Closest Emergent Hospital Time Upper Class,Avg Closest Emergent Hospital Time Statewide
Alabama,13.6417262189878,13.134454586164713,10.42881494316986,13.215949695748643,47.37143685403422,42.78797344593328,27.127365304062305,43.5937010548857
Georgia,11.626011191787278,13.156205545065365,11.286542788071479,12.993670618614036,42.34190244142426,35.80711017892099,26.533818176802615,37.006902464314315
Mississippi,12.16366833145322,11.8953525110849,12.085109239156566,12.038354540788223,62.812383755044245,54.99078890959644,22.48010126278035,58.32138179260404


In [ ]:
--Median by ADI_Rank (Note that non ranked tracts are included in statewide findings)
select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI_Rank='00: Lower Class' then [Closest Hospital Time] end) OVER (PARTITION BY State_Name) AS [Median Closest Hospital Time Low Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI_Rank='01: Middle Class' then [Closest Hospital Time] end) OVER (PARTITION BY State_Name) AS [Median Closest Hospital Time Middle Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI_Rank='02: Upper Class' then [Closest Hospital Time] end) OVER (PARTITION BY State_Name) AS [Median Closest Hospital Time Upper Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY [Closest Hospital Time]) OVER (PARTITION BY State_Name) AS [Median Closest Hospital Time Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI_Rank='00: Lower Class' then [CLOSEST EMERGENT TIME] end) OVER (PARTITION BY State_Name) AS [Median Closest Emergent Hospital Time Low Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI_Rank='01: Middle Class' then [CLOSEST EMERGENT TIME] end) OVER (PARTITION BY State_Name) AS [Median Closest Emergent Hospital Time Middle Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI_Rank='02: Upper Class' then [CLOSEST EMERGENT TIME] end) OVER (PARTITION BY State_Name) AS [Median Closest Emergent Hospital Time Upper Class]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY [CLOSEST EMERGENT TIME]) OVER (PARTITION BY State_Name) AS [Median Closest Emergent Hospital Time Statewide]
from #Final_Map_Table
order by State_Name;


(3 rows affected)

Total execution time: 00:00:00.261

State_Name,Median Closest Hospital Time Low Class,Median Closest Hospital Time Middle Class,Median Closest Hospital Time Upper Class,Median Closest Hospital Time Statewide,Median Closest Emergent Hospital Time Low Class,Median Closest Emergent Hospital Time Middle Class,Median Closest Emergent Hospital Time Upper Class,Median Closest Emergent Hospital Time Statewide
Alabama,10.98792144101149,11.991010362358974,9.454124783310458,11.601299666024637,47.96709138930709,40.05400982921607,23.34772102008428,41.907697256618754
Georgia,9.906141672374396,11.627035407272253,10.875066129301777,11.037669217700714,36.26513089970807,28.05995481130229,18.5186046811815,27.26978209230606
Mississippi,11.02537597176023,11.344380906109901,11.198731303556237,11.154917010340867,62.1542398844082,58.962487160542594,18.218497794508757,59.19426063772927


In [ ]:
--Export tables to disk so that it can connect to Tableau and ArcGIS
select distinct *
into [HCAN361_25].[dbo].[Final_Map_Table_Group1]
from #Final_Map_Table;

alter table [HCAN361_25].[dbo].[Final_Map_Table_Group1]
alter column GEOID_Data varchar(25) not null;

alter table [HCAN361_25].[dbo].[Final_Map_Table_Group1]
add constraint pk_G1_Map Primary Key(GEOID_Data);

select distinct *
into [HCAN361_25].[dbo].[PROV_LOCS_Group1]
from #PROV_LOCS;

alter table [HCAN361_25].[dbo].[PROV_LOCS_Group1]
alter column ID varchar(20) not null;

alter table [HCAN361_25].[dbo].[PROV_LOCS_Group1]
add constraint pk_G1_Prov Primary Key (ID);

(3814 rows affected)

(103 rows affected)

Total execution time: 00:00:00.267